# DSPy Workshop

In [0]:
%pip install -q dspy mlflow
%restart_python

### Setup MLflow Tracing

Let's set up MLflow tracing for the entire lab for interpretability. This lab is prepared with Databricks managed MLflow service, so we don't need to handle the MLflow server management. If you are running the notebook on your local machine, you will need to spin up your own MLflow server. To do so, make sure you have `mlflow` installed, then execute the command below in your terminal:

```bash
mlflow ui  --host 127.0.0.1 --port 8080
```

Then add the following line to the jupyter notebook:

```python
mlflow.set_tracking_uri("http://127.0.0.1:8080")
```

Then you can run the following lab without issues! If you are having trouble setting up the local MLflow server, please refer to the [MLflow Quickstart](https://mlflow.org/docs/latest/getting-started/intro-quickstart/)

In [0]:
import mlflow

mlflow.dspy.autolog()

## DSPy Backbone - Signature and Modules

In this lab, we will demonstrate the fundamentals of DSPy by walking through an end to end demo of building a DSPy program. You will learn how to set the LM for the program, and use DSPy signature to define your input/output data and format, then build a DSPy program with dspy modules to achieve your task. You will also learn how to save/load your DSPy program for future usage and sharing.

In [0]:
import dspy
import os

os.environ["OPENAI_API_KEY"] = "{your_openai_api_key}"

In [0]:
import litellm
import logging

litellm._logging._disable_debugging()
httpx_logger = logging.getLogger("httpx")
httpx_logger.setLevel(logging.WARNING)

### Configure the LM

Configure the LM behind our DSPy workflow. The identifier is of the format `{provider_name}/{lm_name}`, e.g., `databricks/databricks-claude-3-7-sonnet` or `openai/gpt-4o` To find the the complete list of provider and supported models, please refer to the [Litellm documentation](https://docs.litellm.ai/docs/providers).

In this lab, we will be using the `openai/gpt-4o-mini` model. If you decide to switch to a different LM in the future, simply call `dspy.settings.configure(lm=dspy.LM({new_lm_identifier}))`, and the rest of the code can stay unchanged!

In [0]:
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Use DSPy built-in Module to Build a Sentiment Classifier

Now let's start simple by building a sentiment classifier with a single DSPy built-in module. The goal is to classify the text into sentiment of a scale of 10. A higher score means a more positive sentiment.

### Build the Signature to Define Input/Output Data and Format

The next step is creating a signature to define your input/output data and format. We have two different options:

- String-based signature: easy to authorize, but not suitable for complex use cases.
- Class-based signature: powerful, slightly more complex than string-based signature.

Let's explore both options.

#### String-based Signature

String-based signature separates input and output fields by `->`, and separates each field by `,`. For example, `question, context -> answer` means the input has 2 fields, `question` and `context`, and the output has a single field `answer`. To use a string-based signature, we can use `dspy.make_signature()`, or directly feed it to a DSPy module, e.g., `dspy.Predict("question, context -> answer")`. 

Below we create the string-based signature for sentiment classifier.

In [0]:
str_signature = dspy.make_signature("text -> sentiment")

Class-based Signature

To use class-based signature, we need to define a class subclassing from `dspy.Signature`, and configure input and output fields as class attrs, similar to [Pydantic model](https://docs.pydantic.dev/latest/concepts/models/) if you are familar with it.

In [0]:
class SentimentClassifier(dspy.Signature):
    """Classify the sentiment of a text."""

    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(desc="sentiment, the higher the more positive", ge=0, le=10)

In order to distinguish between input fields and output fields, simply assign `dspy.InputField` to input fields, and `dspy.OutputField` to output fields. Fields support the following custom configs:

- type: you can specify the type of the field using format `: {type}`, similar as Python type hint.
- desc: a string explaining the purpose of this field. 
- pydantic constraints: additional constraint on the fields, such as `ge` means must be greater or equal to.

Please refer to [signature tutorial](https://dspy.ai/learn/programming/signatures/) for a thorough guide on defining a signature.

### Create a Module to Interact with the LM

Now we have defined the input/output by creating a signature, the next step is to call LM to classify the sentiment of user text. We can simply pass the signature to the very basic DSPy module `dspy.Predict` to get the job done.

In [0]:
predict = dspy.Predict(SentimentClassifier)

You can also use `predict = dspy.Predict(str_signature)` or `predict = dspy.Predict("text -> sentiment")` to replace the code above. 

Now you can interact with the LM by calling the `predict` with `text` as the input.

In [0]:
output = predict(text="I am feeling pretty happy!")
print(output)

Prediction(
    sentiment=8
)


Trace(request_id=tr-2944b148f1044a339f4bf2996a098a93)

The output is a `dspy.Prediction` instance with a single field `sentiment` of type int (as requested in the signature). You can directly access the field by dot access, or dict access.

In [0]:
print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is: {output['sentiment']}")

The sentiment is: 8
The sentiment is: 8


### Wait, Where is My Prompt? 

It's a bit counterintuitive that with the code above, we never compose the multiturn prompts as requested by LLM providers, e.g., OpenAI, Anthropic, Google and so on. Instead, we are just making calls to a python callable. The magic is under the hood we format the signature together with user input into the multiturn prompt, and we can inspect it by utilizing the util function `dspy.inspect_history()`.

In [0]:
dspy.inspect_history(n=1)





[2025-04-22T22:13:39.300194]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with the corresponding output fields, starting with the field `[[ ## sentiment ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## sentiment ## ]]
8

[[ ## completed ## ]]







In the above console print, the "System message: " section is the the message of role `system`, and "User message: " is the message of role `user`. "Response: " section is the actual response we get from the LLM.

### Try a Different Built-in Module

We have tried out the very basic module `dspy.Predict` above, let's try something more powerful. Another common built-in module we recommend is the `dspy.ChainOfThought`, which is almost the same as `dspy.Predict`, but in addition to the normal `dspy.Predict`'s output, we also request LM to provide `reasoning` for its response. Let's check it out.

In [0]:
cot = dspy.ChainOfThought(SentimentClassifier)

output = cot(text="As a Mavericks fan, I am feeling a bit frustrated about Luka gets traded...")
print(output)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Prediction(
    reasoning='The text expresses frustration regarding the potential trade of a beloved player, Luka, which indicates a negative sentiment. The use of the word "frustrated" clearly conveys disappointment and concern, leading to a lower sentiment score.',
    sentiment=3
)


Trace(request_id=tr-a561bf8f251d4e59a23a04a3c52972a4)

You can see above that the output includes a `reasoning` field explaining why we reach this sentiment score. Now let's check the LM interaction history.

In [0]:
dspy.inspect_history(n=1)





[2025-04-22T22:13:49.858819]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str)
2. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
As a Mavericks fan, I am feeling a bit frustrated about Luka gets traded...

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## sentiment ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## complete

Comparing it to the history of `dspy.Predict`, we can see that with `dspy.ChainOfThought` we are explicitly requesting reasoning from the LM. 

For information of other pre-built modules, please refer to the [API Reference](https://dspy.ai/api/modules/BestOfN/).


### [Optional] Use a Different Adapter

Sometimes the default adapter may not be able to have LLM provide the requested output fields, when it happens you can try a different adapter. To change the adapter, we can make a call to `dspy.configure` like below, where we switch the adapter to be `dspy.JSONAdapter()`, which leverages the structured output feature of the provider. 

In [0]:
dspy.configure(adapter=dspy.JSONAdapter())

Let's interact with LM again and see what's getting changed.

In [0]:
print(cot(text="As a Lakers fan, I feel thrilled of getting Luka Doncic"))
dspy.inspect_history(n=1)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Prediction(
    reasoning="The text expresses excitement and happiness about the acquisition of Luka Doncic, indicating a strong positive sentiment. The use of the word 'thrilled' suggests a high level of enthusiasm and satisfaction, which contributes to a positive sentiment score.",
    sentiment=9
)




[2025-04-22T22:14:04.443384]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str)
2. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

[[ ## reasoning ## ]]
{reasoning}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value
In adhering to this structure, your objec

Trace(request_id=tr-64bc28e80a824c6293ef0750b6409399)

By a careful comparison, you can see that the new Response comes in the format of a dictionary (json) instead of a string broken down into a few sections.

## Build a Program with Custom Module

So far we have explored how to use built-in DSPy module to build a single-module DSPy program. In actual production cases, we usually need to build multi-module programs which also involves custom function calling. Let's get started with it!

### Define Your Program by Customizing DSPy Module

In order to define a custom DSPy program, you will need to define a class subclassing from `dspy.Module`, and write your custom logic in the `forward` method. This is similar PyTorch and Keras if you are familiar with those.

For demo, let's build a celebrity guessing game with DSPy. The game setup is:
- User is asked to think about a celebrity name, and keep it in mind.
- LLM starts by asking a series of yes/no questions. User must response with yes/no. 
- When the max number of questions is reached, LLM must make its guess, and user needs to response to if that's correct.
- LLM will tell users about its reasoning process.

In [0]:
class QuestionGenerator(dspy.Signature):
    """Generate a yes or no question in order to guess the celebrity name in users' mind. You can ask in general or directly guess the name if you think the signal is enough. You should never ask the same question in the past_questions."""
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can help narrow down the celebrity name")
    guess_made: bool = dspy.OutputField(desc="If the new_question is the celebrity name guess, set to True, if it is still a general question set to False")


class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the celebrity name in user's mind")
    final_guessor_question: str = dspy.InputField(desc="the final guess or question LM made")
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OutputField(
        desc="reflection on the guessing process, including what was done well and what can be improved"
    )

def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").strip().lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")

class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_tries = 20

    def forward(self):
        celebrity_name = input("Please think of a celebrity name, once you are ready, type the name and press enter...")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers,
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't guess it right.")

        reflection = self.reflection(
            correct_celebrity_name=celebrity_name,
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)


In [0]:
celebrity_guess = CelebrityGuess()

celebrity_guess()

Please think of a celebrity name, once you are ready, type the name and press enter... Lebron James

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Is the celebrity you are thinking of an actor or actress? (y/n):  n

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Is the celebrity you are thinking of a musician? (y/n):  n

...

Is the celebrity you are thinking of LeBron James? (y/n):  y

Yay! I got it right!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The guessing process was well-structured, starting with broad categories and progressively narrowing down to specific traits related to the celebrity. The questions were relevant and guided the guessing effectively. One area for improvement could be to include a question about the celebrity's achievements or specific teams they are associated with, which could further refine the guessing process in future interactions.


Trace(request_id=tr-861d7ce35195430d8f1fa026f3cbed39)

That's it! With DSPy, it's very easy to combine custom logic with multiple LLM calls to build a compound AI system.

## Save and Load

DSPy provides functionality to save/load your program effortlessly. Simply use the `Module.save(path)` method to save your model, which has two modes:
- State-only: only save the internal states into a json file.
- Entire program: save the whole program's information in a pickle file together with metadata like dspy version in a json file. 

### State-only Saving

In [0]:
celebrity_guess.save("dspy_program/celebrity.json", save_program=False)

You can load the state back from an instance of the same type. This is useful if we have changed the internal state of the module. In this use case it's a no-op.

In [0]:
celebrity_guess.load("dspy_program/celebrity.json")

## Whole-program Saving

In [0]:
celebrity_guess.save("dspy_program/celebrity/", save_program=True)

You can load it back by `dspy.load()`, and it will create a new instance the same as `celebrity_guess`.

In [0]:
loaded = dspy.load("dspy_program/celebrity/")

## Debug DSPy with MLflow Tracing

In this section, we will demonstrate how to use mlflow tracing to help visualize and debug DSPy program. As of the demo, we will build a tool calling agent with DSPy ReAct module. ReAct stands for reasoning and acting, which uses LLM to determine the next tool to call, then execute the tool and send the execution result back to LLM to decide if more tools calls are required or return the answer. 

### Setup MLflow Tracing

Let's first install MLflow and set up MLflow tracing.

### Build a Airline Customer Service Agent with DSPy

Let's build a simple airline customer service agent that helps user automatically manage the flight booking, which can do the following:

- Booking new trips.
- Modify existing trips, including flight change and cancellation. 

To build this agent, we need to interact with the following tools:

- `fetch_flight_info`: get flight information for certain dates. 
- `fetch_itinerary`: get the information about booked itinerary. 
- `book_itinerary`: book a flight on behalf of the user.
- `modify_itinerary`: modify the iternary, either flight change or cancellation.
- `get_user_info`: get users' information.
- `file_ticket`: file a backlog ticket to have human assist. 

Let's define the tools. 

In [0]:
from pydantic import BaseModel

class Date(BaseModel):
    # Somehow LLM is bad at specifying `datetime.datetime`
    year: int
    month: int
    day: int
    hour: int

class UserProfile(BaseModel):
    user_id: str
    name: str
    email: str

class Flight(BaseModel):
    flight_id: str
    date_time: Date
    origin: str
    destination: str
    duration: float
    price: float

class Itinerary(BaseModel):
    confirmation_number: str
    user_profile: UserProfile
    flight: Flight

class Ticket(BaseModel):
    user_request: str
    user_profile: UserProfile


Let's make a few fake data for the demo.

In [0]:
user_database = {
    "Adam": UserProfile(user_id="1", name="Adam", email="adam@gmail.com"),
    "Bob": UserProfile(user_id="2", name="Bob", email="bob@gmail.com"),
    "Chelsie": UserProfile(user_id="3", name="Chelsie", email="chelsie@gmail.com"),
    "David": UserProfile(user_id="4", name="David", email="david@gmail.com"),
}

flight_database = {
    "DA123": Flight(
        flight_id="DA123",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=1),
        duration=3,
        price=200,
    ),
    "DA125": Flight(
        flight_id="DA125",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=7),
        duration=9,
        price=500,
    ),
    "DA456": Flight(
        flight_id="DA456",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=1),
        duration=2,
        price=100,
    ),
    "DA460": Flight(
        flight_id="DA460",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=9),
        duration=2,
        price=120,
    ),
}

itinery_database = {}
ticket_database = {}

Now let's define the tools.

In [0]:
import random
import string


def fetch_flight_info(date: Date, origin: str, destination: str):
    """Fetch flight information from origin to destination on the given date"""
    flights = []

    for flight_id, flight in flight_database.items():
        if (
            flight.date_time.year == date.year
            and flight.date_time.month == date.month
            and flight.date_time.day == date.day
            and flight.origin == origin
            and flight.destination == destination
        ):
            flights.append(flight)
    return flights


def fetch_itinerary(confirmation_number: str):
    """Fetch a booked itinerary information from database"""
    return itinery_database.get(confirmation_number)


def pick_flight(flights: list[Flight]):
    """Pick up the best flight that matches users' request."""
    sorted_flights = sorted(
        flights,
        key=lambda x: (
            x.get("duration") if isinstance(x, dict) else x.duration,
            x.get("price") if isinstance(x, dict) else x.price,
        ),
    )
    return sorted_flights[0]


def generate_id(length=8):
    chars = string.ascii_lowercase + string.digits
    return "".join(random.choices(chars, k=length))


def book_itinerary(flight: Flight, user_profile: UserProfile):
    """Book a flight on behalf of the user."""
    confirmation_number = generate_id()
    while confirmation_number in itinery_database:
        confirmation_number = generate_id()
    itinery_database[confirmation_number] = Itinerary(
        confirmation_number=confirmation_number,
        user_profile=user_profile,
        flight=flight,
    )
    return confirmation_number, itinery_database[confirmation_number]


def cancel_itinerary(confirmation_number: str, user_profile: UserProfile):
    """Cancel an itinerary on behalf of the user."""
    if confirmation_number in itinery_database:
        del itinery_database[confirmation_number]
        return
    raise ValueError("Cannot find the itinerary, please check your confirmation number.")


def get_user_info(name: str):
    """Fetch the user profile from database with given name."""
    return user_database.get(name)


def file_ticket(user_request: str, user_profile: UserProfile):
    """File a customer support ticket if this is something the agent cannot handle."""
    ticket_id = generate_id(length=6)
    ticket_database[ticket_id] = Ticket(
        user_request=user_request,
        user_profile=user_profile,
    )
    return ticket_id


The next step is to define the DSPy program, which takes in a user request, and return the process result. 

In [0]:
class DSPyAirlineCustomerSerice(dspy.Signature):
    """You are an airline customer service agent. You are given a list of tools to handle user request. You should decide the right tool to use in order to fullfil users' request."""
    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(desc="Message that summarizes the process result, and the information users need, e.g., the confirmation_number if it's a flight booking request.")

In [0]:
react = dspy.ReAct(
    DSPyAirlineCustomerSerice,
    tools = [
        fetch_flight_info,
        fetch_itinerary,
        pick_flight,
        book_itinerary,
        cancel_itinerary,
        get_user_info,
        file_ticket,
    ]
)

In [0]:
result = react(user_request="please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Trace(request_id=tr-eeff5fe411a3460b9fc6dc4f2584d9dd)

## DSPy Magic - Automatic Program Optimization

In this section, we will explore how to optimize our DSPy program with DSPy optimizer. 

In [0]:
os.environ["HF_DATASETS_CACHE"] = "."

### Define the Evaluate Metrics

Similar to training a machine learning model, we need to define a metric to represent how well our program functions. In this example, we are tackling a math Q&A task, and the metric we use is "equivalence", which means the LM generated answer is equivalent to the golden answer.

In [0]:
def _fix_fracs(string):
    substrs = string.split("\\frac")
    new_str = substrs[0]
    if len(substrs) > 1:
        substrs = substrs[1:]
        for substr in substrs:
            new_str += "\\frac"
            if substr[0] == "{":
                new_str += substr
            else:
                try:
                    assert len(substr) >= 2
                except:
                    return string
                a = substr[0]
                b = substr[1]
                if b != "{":
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}{" + b + "}" + post_substr
                    else:
                        new_str += "{" + a + "}{" + b + "}"
                else:
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}" + b + post_substr
                    else:
                        new_str += "{" + a + "}" + b
    string = new_str
    return string


def _fix_a_slash_b(string):
    if len(string.split("/")) != 2:
        return string
    a = string.split("/")[0]
    b = string.split("/")[1]
    try:
        a = int(a)
        b = int(b)
        assert string == "{}/{}".format(a, b)
        new_string = "\\frac{" + str(a) + "}{" + str(b) + "}"
        return new_string
    except:
        return string


def _remove_right_units(string):
    # "\\text{ " only ever occurs (at least in the val set) when describing units
    if "\\text{ " in string:
        splits = string.split("\\text{ ")
        assert len(splits) == 2
        return splits[0]
    else:
        return string


def _fix_sqrt(string):
    if "\\sqrt" not in string:
        return string
    splits = string.split("\\sqrt")
    new_string = splits[0]
    for split in splits[1:]:
        if split[0] != "{":
            a = split[0]
            new_substr = "\\sqrt{" + a + "}" + split[1:]
        else:
            new_substr = "\\sqrt" + split
        new_string += new_substr
    return new_string


def _strip_string(string):
    # linebreaks
    string = string.replace("\n", "")
    # print(string)

    # remove inverse spaces
    string = string.replace("\\!", "")
    # print(string)

    # replace \\ with \
    string = string.replace("\\\\", "\\")
    # print(string)

    # replace tfrac and dfrac with frac
    string = string.replace("tfrac", "frac")
    string = string.replace("dfrac", "frac")
    # print(string)

    # remove \left and \right
    string = string.replace("\\left", "")
    string = string.replace("\\right", "")
    # print(string)

    # Remove circ (degrees)
    string = string.replace("^{\\circ}", "")
    string = string.replace("^\\circ", "")

    # remove dollar signs
    string = string.replace("\\$", "")

    # remove units (on the right)
    string = _remove_right_units(string)

    # remove percentage
    string = string.replace("\\%", "")
    string = string.replace("\%", "")

    # " 0." equivalent to " ." and "{0." equivalent to "{." Alternatively, add "0" if "." is the start of the string
    string = string.replace(" .", " 0.")
    string = string.replace("{.", "{0.")
    # if empty, return empty string
    if len(string) == 0:
        return string
    if string[0] == ".":
        string = "0" + string

    # to consider: get rid of e.g. "k = " or "q = " at beginning
    if len(string.split("=")) == 2:
        if len(string.split("=")[0]) <= 2:
            string = string.split("=")[1]

    # fix sqrt3 --> sqrt{3}
    string = _fix_sqrt(string)

    # remove spaces
    string = string.replace(" ", "")

    # \frac1b or \frac12 --> \frac{1}{b} and \frac{1}{2}, etc. Even works with \frac1{72} (but not \frac{72}1). Also does a/b --> \\frac{a}{b}
    string = _fix_fracs(string)

    # manually change 0.5 --> \frac{1}{2}
    if string == "0.5":
        string = "\\frac{1}{2}"

    # NOTE: X/Y changed to \frac{X}{Y} in dataset, but in simple cases fix in case the model output is X/Y
    string = _fix_a_slash_b(string)

    return string


def is_equiv(str1, str2, verbose=False):
    if str1 is None and str2 is None:
        print("WARNING: Both None")
        return True
    if str1 is None or str2 is None:
        return False

    try:
        ss1 = _strip_string(str1)
        ss2 = _strip_string(str2)
        if verbose:
            print(ss1, ss2)
        return ss1 == ss2
    except:
        return str1 == str2


def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx == None:
        retval = None
    else:
        retval = string[idx : right_brace_idx + 1]

    return retval


def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[: len(left)] == left
        assert s[-1] == "}"
        return s[len(left) : -1]
    except:
        return None


def remove_format(s):
    # remove \(\)
    s = s.replace("\\(", "")
    s = s.replace("\\)", "")
    return s


def math_evaluate(gold, pred, trace=None):
    return is_equiv(
        remove_boxed(last_boxed_only_string(gold.solution)),
        remove_format(pred.answer),
        verbose=False,
    )

### Convert the Dataset into a List of dspy.Example

In order for the optimizer to work smoothly, it's recommended to convert each data record into a `dspy.Example`. 

In [0]:
from datasets import load_dataset

dataset_math = load_dataset("DigitalLearningGmbH/MATH-lighteval", "default")
math_train = [
    dspy.Example(**x).with_inputs("problem") for x in dataset_math["train"]
]
math_val = [
    dspy.Example(**x).with_inputs("problem") for x in dataset_math["test"]
]

/databricks/python_shell/lib/dbruntime/huggingface_patches/datasets.py:49: UserWarning: The cache_dir for this dataset is ., which is not a persistent path.Therefore, if/when the cluster restarts, the downloaded dataset will be lost.The persistent storage options for this workspace/cluster config are: [UC Volumes].Please update either `cache_dir` or the environment variable `HF_DATASETS_CACHE`to be under one of the following root directories: ['/Volumes/']
  warnings.warn(warning_message)
/databricks/python_shell/lib/dbruntime/huggingface_patches/datasets.py:18: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


In [0]:
math_program = dspy.ChainOfThought("problem->answer")

### Track Optimization with MLflow

We can track DSPy optimization process with mlflow autologging, to enable it, we need to configure a few flags.

In [0]:
import mlflow

mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [0]:
from dspy.evaluate import Evaluate

math_evaluator = Evaluate(devset=math_val[:100], num_threads=10, display_progress=True, display_table=20)

In [0]:
math_eval_score_baseline = math_evaluator(math_program, metric=math_evaluate)

Average Metric: 80.00 / 100 (80.0%): 100%|██████████| 100/100 [00:21<00:00,  4.76it/s]

2025/04/22 22:22:48 INFO dspy.evaluate.evaluate: Average Metric: 80 / 100 (80.0%)
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


,problem,level,solution,type,reasoning,answer,math_evaluate
0,How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6...,Level 3,The denominator of the rational function factors into $x^2+x-6=(x-...,Algebra,To find the vertical asymptotes of the function \( y = \frac{2}{x^...,2,✔️ [True]
1,What is the positive difference between $120\%$ of 30 and $130\%$ ...,Level 1,One hundred twenty percent of 30 is $120\cdot30\cdot\frac{1}{100}=...,Algebra,To find the positive difference between $120\%$ of 30 and $130\%$ ...,10,✔️ [True]
2,Find $x$ such that $\lceil x \rceil + x = \dfrac{23}{7}$. Express ...,Level 4,"First, we note that $x$ must be positive, since otherwise $\lceil ...",Algebra,Let \( x = n + f \) where \( n = \lfloor x \rfloor \) (the greates...,\frac{2}{7},
3,Evaluate $i^5+i^{-25}+i^{45}$.,Level 5,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. We also have $i^{-25}...,Algebra,"To evaluate the expression $i^5 + i^{-25} + i^{45}$, we first need...",i,✔️ [True]
4,"If $2^8=4^x$, what is the value of $x$?",Level 1,"Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Since $2^8=2^{2x}$, we...",Algebra,"To solve the equation \(2^8 = 4^x\), we can express \(4\) in terms...",4,✔️ [True]
5,"What is the 100th term of the arithmetic sequence 6, 10, 14, 18, ...?",Level 2,"The common difference is $10 - 6 = 4$, so the 100th term is $6+99\...",Algebra,"In an arithmetic sequence, the nth term can be calculated using th...",402,✔️ [True]
6,For what values of $x$ is it true that $x^2 - 5x - 4 \le 10$? Expr...,Level 4,"Re-arranging, $x^2 - 5x - 14 \le 0$. The left-hand quadratic facto...",Algebra,"To solve the inequality \( x^2 - 5x - 4 \le 10 \), we first rearra...","[-2, 7]",
7,Mr. Madoff invests 1000 dollars in a fund that compounds annually ...,Level 4,"Let $r$ be the annual interest rate. Then after three years, Mr. M...",Algebra,"To find the annual interest rate, we can use the formula for compo...",8,
8,"Four distinct integers $a$, $b$, $c$ and $d$ have the property tha...",Level 4,"WLOG, let $a<b<c<d$. The smallest sum is $a+b=10$. The second-smal...",Algebra,"Let the four distinct integers be denoted as a, b, c, and d. The s...","4, 6, 14, 15",✔️ [True]
9,What is the smallest value of $x$ such that $|5x - 1| = |3x + 2|$?...,Level 5,"There are two cases, when $5x-1=3x+2$ and when $5x-1=-(3x+2).$ The...",Algebra,"To solve the equation |5x - 1| = |3x + 2|, we need to consider the...",-\frac{1}{8},✔️ [True]


INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


[Trace(request_id=tr-0cbe9e72c90a4aa7afab179c090c6b21), Trace(request_id=tr-2cf5ffe35df742c0aced23f86c595d5b), Trace(request_id=tr-9b31230805924151b157855539365e42), Trace(request_id=tr-1350ad22e27140b893cd064a6d31d18a), Trace(request_id=tr-7c565b2aba0c4b75824d0b93168758e8), Trace(request_id=tr-78d669c8f11a49a69dd5d8613494bd40), Trace(request_id=tr-b39318de04984f45a6f8e84d275b902f), Trace(request_id=tr-3d518355b32e4d71845b1aabeb3e69f9), Trace(request_id=tr-d142aadb684c46a6973c5468f0191ca5), Trace(request_id=tr-dba0f58f19b846b7b110326c6f63509c)]

Check out a sample data input/output.

In [0]:
math_program(**math_train[0].inputs())

Prediction(
    reasoning='To ensure that the piecewise function \\( f(x) \\) is continuous, we need to check the continuity at the points where the definition of the function changes, which are at \\( x = -2 \\) and \\( x = 2 \\).\\n\\n1. **Continuity at \\( x = 2 \\)**:\\n   - From the left (using the second piece): \\( f(2) = 2 - 5 = -3 \\)\\n   - From the right (using the first piece): \\( f(2) = a(2) + 3 = 2a + 3 \\)\\n   - For continuity at \\( x = 2 \\): \\( 2a + 3 = -3 \\)\\n   - Solving for \\( a \\): \\( 2a = -6 \\) \\( \\Rightarrow a = -3 \\)\\n\\n2. **Continuity at \\( x = -2 \\)**:\\n   - From the left (using the third piece): \\( f(-2) = 2(-2) - b = -4 - b \\)\\n   - From the right (using the second piece): \\( f(-2) = -2 - 5 = -7 \\)\\n   - For continuity at \\( x = -2 \\): \\( -4 - b = -7 \\)\\n   - Solving for \\( b \\): \\( -b = -3 \\) \\( \\Rightarrow b = 3 \\)\\n\\nNow we have \\( a = -3 \\) and \\( b = 3 \\). Therefore, \\( a + b = -3 + 3 = 0 \\).',
    answer='0'


Trace(request_id=tr-1708a05f716e44f1911bda3a2d97174a)

### Configure the Optimizer and Kick Off Optimization

The last step is configuring the optimizer, and kick off the optimization process. Here we are using `MIPROv2` optimizer, which optimizes both the `instruction` (system prompt) and `demos` (few-shot examples).

In [0]:
mipro = dspy.MIPROv2(
    metric=math_evaluate,
    max_bootstrapped_demos=4,
    max_labeled_demos=4,
    num_candidates=4,
    auto=None,
)

optimized_math_program = mipro.compile(
    math_program,
    trainset=math_train[:100],
    valset=math_train[100:200],
    num_trials=8,
    minibatch_size=4,
    minibatch_full_eval_steps=4,
)

2025/04/22 22:23:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f3e835cef9f8448c81be5c8966ee62d5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow


Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data ... y

2025/04/22 22:23:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/04/22 22:23:44 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/04/22 22:23:44 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=4 sets of demonstrations...


Bootstrapping set 1/4
Bootstrapping set 2/4
Bootstrapping set 3/4


  4%|▍         | 4/100 [00:15<06:23,  4.00s/it]


...

Average Metric: 85.00 / 100 (85.0%): 100%|██████████| 100/100 [02:06<00:00,  1.26s/it]

2025/04/22 22:33:26 INFO dspy.evaluate.evaluate: Average Metric: 85 / 100 (85.0%)
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


2025/04/22 22:33:27 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [86.0, 88.0, 85.0]
2025/04/22 22:33:27 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 88.0
2025/04/22 22:33:27 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/04/22 22:33:27 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/04/22 22:33:27 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 88.0!


[Trace(request_id=tr-8f164c632cdf436aa52689fd7372caa8), Trace(request_id=tr-d6521015b172418185418907027c1fe3), Trace(request_id=tr-66b5fd4cd8384d92840fbbe3c729a8cb), Trace(request_id=tr-464ac7f28329469683491402a081bbc4), Trace(request_id=tr-9d0b396ce34b461cbf537083644e175d), Trace(request_id=tr-1f60c0b657134372a90b97bdbc2a2f74), Trace(request_id=tr-90c739c0fce64625be49444681564bbb), Trace(request_id=tr-6acd7ad632654e059b80d1e5a4051262), Trace(request_id=tr-daddec08928242c58fe0599d3f9a77da), Trace(request_id=tr-5553bd0896c4437b9fbd2fbd584eec61)]

INFO:py4j.clientserver:Closing down clientserver connection


Check out the performance of the optimized program.

In [0]:
math_eval_score_optimized = math_evaluator(optimized_math_program, metric=math_evaluate)

Average Metric: 24.00 / 31 (77.4%):  31%|███       | 31/100 [00:36<01:37,  1.41s/it]

2025/04/22 23:34:00 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning'])


Average Metric: 30.00 / 40 (75.0%):  40%|████      | 40/100 [00:46<00:51,  1.16it/s]

2025/04/22 23:34:11 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning'])


Average Metric: 41.00 / 54 (75.9%):  54%|█████▍    | 54/100 [01:00<00:57,  1.26s/it]

...

2025/04/22 23:35:04 INFO dspy.evaluate.evaluate: Average Metric: 80.0 / 100 (80.0%)
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


,problem,level,solution,type,reasoning,answer,math_evaluate
0,How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6...,Level 3,The denominator of the rational function factors into $x^2+x-6=(x-...,Algebra,To find the vertical asymptotes of the function \( y = \frac{2}{x^...,2,✔️ [True]
1,What is the positive difference between $120\%$ of 30 and $130\%$ ...,Level 1,One hundred twenty percent of 30 is $120\cdot30\cdot\frac{1}{100}=...,Algebra,"To find the positive difference between 120% of 30 and 130% of 20,...",10,✔️ [True]
2,Find $x$ such that $\lceil x \rceil + x = \dfrac{23}{7}$. Express ...,Level 4,"First, we note that $x$ must be positive, since otherwise $\lceil ...",Algebra,"Let \( x = n + f \), where \( n = \lceil x \rceil \) is an integer...",\frac{16}{7},
3,Evaluate $i^5+i^{-25}+i^{45}$.,Level 5,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. We also have $i^{-25}...,Algebra,"To evaluate the expression \( i^5 + i^{-25} + i^{45} \), we first ...",i,✔️ [True]
4,"If $2^8=4^x$, what is the value of $x$?",Level 1,"Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Since $2^8=2^{2x}$, we...",Algebra,"To solve the equation \( 2^8 = 4^x \), we first express \( 4 \) in...",4,✔️ [True]
5,"What is the 100th term of the arithmetic sequence 6, 10, 14, 18, ...?",Level 2,"The common difference is $10 - 6 = 4$, so the 100th term is $6+99\...",Algebra,"In an arithmetic sequence, the nth term can be calculated using th...",402,✔️ [True]
6,For what values of $x$ is it true that $x^2 - 5x - 4 \le 10$? Expr...,Level 4,"Re-arranging, $x^2 - 5x - 14 \le 0$. The left-hand quadratic facto...",Algebra,"To solve the inequality \( x^2 - 5x - 4 \le 10 \), we first rearra...","[-2, 7]",
7,Mr. Madoff invests 1000 dollars in a fund that compounds annually ...,Level 4,"Let $r$ be the annual interest rate. Then after three years, Mr. M...",Algebra,"To find the annual interest rate, we can use the formula for compo...",8,
8,"Four distinct integers $a$, $b$, $c$ and $d$ have the property tha...",Level 4,"WLOG, let $a<b<c<d$. The smallest sum is $a+b=10$. The second-smal...",Algebra,"Let the four distinct integers be a, b, c, and d. The sums of the ...","4, 6, 14, 15",✔️ [True]
9,What is the smallest value of $x$ such that $|5x - 1| = |3x + 2|$?...,Level 5,"There are two cases, when $5x-1=3x+2$ and when $5x-1=-(3x+2).$ The...",Algebra,"To solve the equation \(|5x - 1| = |3x + 2|\), we need to consider...",-1/8,


INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


[Trace(request_id=tr-17294e7b5fee42a7a6eac609709a5704), Trace(request_id=tr-376cebe0f0ca4effb3f4a0756d207385), Trace(request_id=tr-5401e251236547d083d444c4100177e5), Trace(request_id=tr-528ff0ad71374304b8ccbc3dc4bdb046), Trace(request_id=tr-664e9a7c0cc340c8b94d3f0ad3163601), Trace(request_id=tr-444776b305ee42a58910b626df075600), Trace(request_id=tr-6585cf57e12a4932bf595310162b2122), Trace(request_id=tr-d4340826056947dd9fc9e82d78aff0af), Trace(request_id=tr-118d8a69def645b9af855cff84a5a59d), Trace(request_id=tr-b44a1ba8679d43b9b6f85f60f6be3966)]

In [0]:
optimized_math_program.predict.signature.instructions

'You are a mathematician tasked with solving critical algebraic problems that will determine the fate of a prestigious mathematics competition. Given the problem statement, provide a thorough step-by-step reasoning process leading to the final answer. Make sure to connect theoretical concepts with practical applications, ensuring clarity and rigor in your solution. Use the fields `problem` to guide your response and produce the fields `reasoning` and `answer` accordingly.'

In [0]:
optimized_math_program.predict.demos

[Example({'augmented': True, 'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).', 'reasoning': 'To ensure that the piecewise function \\( f(x) \\) is continuous, we need to check the continuity at the points where the definition of the function changes, which are at \\( x = -2 \\) and \\( x = 2 \\).\\n\\n1. **Continuity at \\( x = 2 \\)**:\\n   - From the left (using the second piece): \\( f(2) = 2 - 5 = -3 \\)\\n   - From the right (using the first piece): \\( f(2) = a(2) + 3 = 2a + 3 \\)\\n   - For continuity at \\( x = 2 \\): \\( 2a + 3 = -3 \\)\\n   - Solving for \\( a \\): \\( 2a = -6 \\) \\( \\Rightarrow a = -3 \\)\\n\\n2. **Continuity at \\( x = -2 \\)**:\\n   - From the left (using the third piece): \\( f(-2) = 2(-2)

In [0]:
# Define a QA module with chain of thought
qa = dspy.ChainOfThought("question -> answer")


# Define a reward function that checks for one-word answers
def one_word_answer(args, pred):
    score = 1.0 if len(pred.answer.split()) < 5 else 0.0
    print(f"Score: {score}")
    return score


# Create a refined module that tries up to 3 times
best_of_3 = dspy.Refine(module=qa, N=3, reward_fn=one_word_answer, threshold=1.0)

# Use the refined module
result = best_of_3(question="What's a good reason for working?").answer

print(result)
